In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
url = "/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book.json"

with open(url,"r",encoding="utf-8") as f:
    data_original = json.load(f)

In [3]:
data_original = pd.DataFrame(data_original)

In [4]:
data_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  object 
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           46967 non-null  object 
 5   제목           49996 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          47002 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [9]:
# 원본 데이터
a = data_original['제목'].astype(str)
b = data_original['책소개'].astype(str)

# \n 전부 제거
a_clean = a.apply(lambda x: re.sub(r'\n+', '', x).strip())
b_clean = b.apply(lambda x: re.sub(r'\n+', '', x).strip())

# 토씨하나까지 동일한 텍스트인가?
exact_match_count = (a_clean == b_clean).sum()

print(f"토씨하나까지 동일한 텍스트 : {exact_match_count}")

토씨하나까지 동일한 텍스트 : 3


In [10]:
matching_rows = data_original[a_clean == b_clean]

In [11]:
matching_rows

# ISBN
# ['9788929512385','9791137231061','9791137249509']
# 9788929512385 만 제목과 책소개가 None
# 나머지는 책소개가 책 제목으로 들어가있음

,ISBN,ITEM_ID,BID,GOODS_NO,분류,제목,부제,원제,저자,발행자,...,페이지,가격,표지,간략소개,책소개,저자소개,목차,출판사리뷰,INSERT_DATE,UPDATE_DATE
25705,9788929512385,250757730,16659237.0,NaN,"[""국내도서 > 어린이 > 어린이 영어 > 어휘/단어"", ""국내도서 > 어린이 > ...",None,디즈니 겨울왕국 사이트 워드,None,(주)블루앤트리 지음,블루앤트리,...,240,39000,https://image.aladin.co.kr/product/25075/77/co...,None,None,None,스토리북 10권\n워크북 10권\n플래시카드 1권(50장),None,2022-03-19T18:16:17.000Z,None
41675,9791137231061,262119111,17806594.0,NaN,"[""국내도서 > 소설/시/희곡 > 시 > 한국시""]",시공초월,None,None,오아율 지음,BOOKK(부크크),...,105,15000,https://image.aladin.co.kr/product/26211/91/co...,시공초월,시공초월,"오아율\n저자 : 오아율\n작가. 뜻, 마음, 생각 함께인 공존.",출발구성\n여름 열기\n온도솜\n달띠\n밤 공기\n둥근 물방울\n삶에 대한 밀착감\n열렬,None,2022-03-21T23:01:38.000Z,None
48154,9791137249509,275621255,20768212.0,NaN,"[""국내도서 > 예술/대중문화 > 연극 > 한국희곡"", ""국내도서 > 소설/시/희곡...",강해원 희곡집,None,None,강해원 지음,부크크(bookk),...,55,5600,https://image.aladin.co.kr/product/27562/12/co...,강해원 희곡집,강해원 희곡집,강해원\n저자 : 강해원\n1997년 출생.,시험\r\n바다가뭘까\r\n병원\r\n비\r\n교환학생\r\n공연\r\n노래\r\n전화,None,2022-03-22T00:04:27.000Z,None


In [12]:
# 원본 데이터
a = data_original['제목'].astype(str)
b = data_original['간략소개'].astype(str)

# \n 전부 제거
a_clean = a.apply(lambda x: re.sub(r'\n+', '', x).strip())
b_clean = b.apply(lambda x: re.sub(r'\n+', '', x).strip())

# 토씨하나까지 동일한 텍스트인가?
exact_match_count = (a_clean == b_clean).sum()

print(f"토씨하나까지 동일한 텍스트 : {exact_match_count}")

토씨하나까지 동일한 텍스트 : 1647


In [13]:
# 해당 데이터는 간략 소개를 책 소개로 대체할 수 없음.
# 따로 ISBN을 뽑아낸 뒤, 데이터를 합칠 때 조건을 달아야 함.
matching_rows = data_original[a_clean == b_clean]
matching_rows

,ISBN,ITEM_ID,BID,GOODS_NO,분류,제목,부제,원제,저자,발행자,...,페이지,가격,표지,간략소개,책소개,저자소개,목차,출판사리뷰,INSERT_DATE,UPDATE_DATE
95,9788928518357,314833138,NaN,118413272.0,"[""국내도서 > 역사 > 한국근현대사 > 근대개화기""]",이재수 실기,“이재수의 난”을 기록하고 아카이브하다,None,"유춘동, 신성환, 박종후 저 지음",민속원,...,192,12000,https://image.aladin.co.kr/product/31483/31/co...,이재수 실기,None,None,머리말<br/><br/>해설. 조무빈趙武彬이 펴낸 『이재수 실기­야월의 한라산』<b...,None,2024-03-21T14:04:30.000Z,None
104,9788915001275,319093872,NaN,119692340.0,"[""국내도서 > 유아 > 4~7세 > 그림책"", ""국내도서 > 유아 > 그림책 > ...",라푼젤,None,None,그림형제 원작 / 최은정 글 / 엘레오노르 델라 말바 그림 지음,삼성출판사,...,23,5900,https://image.aladin.co.kr/product/31909/38/co...,라푼젤,None,None,None,None,2024-05-11T00:54:38.000Z,None
231,9791160455908,319372170,NaN,119809642.0,None,전기기사·전기산업기사 기사실기특강,대한민국 최고강사의 실기 특강 NO.1 전기교재,None,이종칠 저 지음,세진사,...,643,35000,https://image.aladin.co.kr/product/31937/21/co...,전기기사·전기산업기사 기사실기특강,None,저 : 이종칠\n전기공학 박사\n現) 제일전기기술학원 원장\n現) 커넥츠 전기단기 ...,『제1편 예비전원설비』\n01 예비전원설비\n1.1 자가 발전 설비\n1.2 축전지...,None,2024-05-13T12:03:50.000Z,None
250,9788963043593,314410901,NaN,118162474.0,"[""국내도서 > 수험서/자격증 > 공무원 수험서 > 소방공무원(승진) > 소방승진""]",소방법령 1 핵심정리,None,None,최승수 저 지음,다인(반),...,576,30000,https://image.aladin.co.kr/product/31441/9/cov...,소방법령 1 핵심정리,None,None,제1장 총 칙 12<br/>핵심기출문제 및 예상문제 15<br/>제2장 소방공무원인...,None,2024-03-19T02:06:12.000Z,None
256,9791129045935,312780934,NaN,117834456.0,"[""국내도서 > 대학교재/전문서적 > 공학계열 > 건축공학 > 건축사/건축일반""]",건설·시설 안전분야 정보관리체계 개선 및 활용방안 연구,None,None,국토안전관리원 저 지음,진한엠앤비,...,286,28000,https://image.aladin.co.kr/product/31278/9/cov...,건설·시설 안전분야 정보관리체계 개선 및 활용방안 연구,None,None,1. 서론<br/>1.1 연구 배경 및 필요<br/>1.2 연구 목표 및 내<br/...,건설·시설 안전을 위해 국토안전관리원은 건설공사 안전관리 종합정보망(Construc...,2023-03-13T07:39:26.000Z,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49861,9791192746425,318363974,NaN,118843571.0,"[""국내도서 > 고등학교참고서 > 수능모의고사/기출 > 국어영역"", ""국내도서 > ...",수능 마무리 봉투모의고사 파이널 테스트 국어영역 화법과 작문 (2023년),season 3,None,편집부 저 지음,수능적중평가원,...,0,10000,https://image.aladin.co.kr/product/31836/39/co...,수능 마무리 봉투모의고사 파이널 테스트 국어영역 화법과 작문 (2023년),None,None,None,None,2024-05-05T13:15:22.000Z,None
49938,9791169501262,304131754,NaN,115081645.0,"[""국내도서 > 예술/대중문화 > 건축 > 건축이론/비평/역사""]",관공서중심의 공공디자인복합건축물,개정판,None,편집부 저 지음,월드해피북스,...,184,99000,https://image.aladin.co.kr/product/30413/17/co...,관공서중심의 공공디자인복합건축물,None,None,None,None,2022-12-27T10:12:49.000Z,None
49950,9788968472398,314404335,NaN,118269598.0,"[""국내도서 > 대학교재/전문서적 > 의약학간호계열 > 시험 대책 > 간호사 국가 ...",Vaccine 1. 성인간호학 기출문제풀이,2024년 대비,None,북샘터 학술 편찬국 저 지음,북샘터,...,903,42000,https://image.aladin.co.kr/product/31440/43/co...,Vaccine 1. 성인간호학 기출문제풀이,None,None,None,None,2024-03-19T00:04:23.000Z,None
49962,9788956768861,303098275,NaN,108611884.0,"[""국내도서 > 대학교재/전문서적 > 예체능계열 > 체육""]",운동 역학,전정판,None,고규철 등저 지음,대경북스,...,193,17000,https://image.aladin.co.kr/product/30309/82/co...,운동 역학,None,None,제1장 운동역학의 개요<br/>제2장 운동역학의 이해<br/>제3장 인체역학<br/...,None,2023-01-04T18:44:01.000Z,None


In [ ]:
data_original[['제목','간략소개']].iloc[95,:]

,95
제목,이재수 실기
간략소개,이재수 실기


In [ ]:
data_original[['제목','부제','분류']][~data_original['분류'].str.contains('국내도서', na=False)]

,제목,부제,분류
37,합격자 Win-Q 무역영어 1급·2급 단기완성,2022,None
43,2022 위생사 실기시험문제,개정16판,None
87,해커스 세무회계엔딩 소득세법(2020)(개정판),"2020 최신 개정세법 반영ㅣ공인회계사ㆍ세무사 1,2차 시험 대비",None
118,에듀윌 전산회계 1급 이론편+실무편+최신기출,"최대 70강 무료,모든 이론 강의 72시간 무료,2020",None
119,일반기계기사 필기+무료동영상+핸드북,2022,None
...,...,...,...
49928,요양보호사 필기&실기문제집 7일만에 합격하기,2022,None
49932,조경기능사 실기,"NCS 기반 출제기준에 따른, 2022",None
49935,딱!! 한권으로 끝내주는 이용사 필기·실기 시험문제,"이용장 포함, 2022",None
49965,핵심 소방기술사,18판,None


In [14]:
data = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book_250306_update(filled)_v2.csv")

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49206 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          49626 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
data[['제목','책소개']].iloc[95,:]

,95
제목,이재수 실기
책소개,『이재수 실기』는 1901년 제주도에서 발생했던 ‘이재수의 난’의 전 과정을 소설 ...


In [16]:
a = data['제목'].astype(str)
b = data['책소개'].astype(str)

# \n 전부 제거
a_clean = a.apply(lambda x: re.sub(r'\n+', '', x).strip())
b_clean = b.apply(lambda x: re.sub(r'\n+', '', x).strip())

# 토씨하나까지 동일한 텍스트인가?
exact_match_count = (a_clean == b_clean).sum()

matching_rows = data[a_clean == b_clean]

print(f"토씨하나까지 동일한 텍스트 : {exact_match_count}")

토씨하나까지 동일한 텍스트 : 363


In [17]:
filtered_data = matching_rows[matching_rows[['저자소개', '출판사리뷰']].notna().any(axis=1)]
len(filtered_data)

167

In [18]:
filtered_data = matching_rows[matching_rows[['출판사리뷰']].isna().all(axis=1)]
len(filtered_data)

242

### Columns
* ISBN : 국제 표준 도서 번호(필수 식별자)
* ITEM_ID : 데이터베이스 또는 시스템 상에서 책을 구분하는 고유 ID ()
* BID : 서지 데이터에서의 추가적인 ID?
* GOODS_NO : 상품 번호

* -> **ISBN을 제외하고 전부 삭제**
* -> **동일 특수기호는 LLM이 알아먹기 쉽게 전체 프로세스에서 동일한 내재적 의미를 지녀야함**
-> 모든 ">" 기호는 대분류 -> 소분류의 의미

### 분류
* 1. None 794를 수집한 후 데이터 form의 중복없이, 사용자 친화적으로, RAG 내에서 의미적 유사성을 이해하기 쉬운 형태로 변환
    * LLM 활용 (GPT 4o 와 CLOVA X 둘 다 큰 차이는 없음)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [29]:
len(data['분류'].loc[data['분류'].isna()])

794

In [31]:
data[['제목','부제','분류']][data['제목'].str.contains('문제집', na=False)]

,제목,부제,분류
7,내신콘서트 중학 중간+기말고사 기출문제집 영어 중3-1,"미래엔 최연희,2020",['국내도서 > 중학교참고서 > 중3-문제집 > 영어(중등3)']
34,고등 수학(하) 수능기출문제집,"2021,자이스토리","['국내도서 > 고등학교참고서 > 수능모의고사/기출 > 수학영역', '국내도서 > ..."
83,티오피 클래스 T.O.P CLASS 전국연합 학력평가 기출문제집 고등 영어 독해 기본,"SKY 선배들의 문제 풀이 노하우,2022","['국내도서 > 고등학교참고서 > 고등영어전문교재 > 독해/작문', '국내도서 > ..."
144,김형준 직업상담심리학개론 영역&실전 문제집,2022,['국내도서 > 수험서/자격증 > 공무원 수험서 > 7/9급 공무원 > 직군별 문제...
193,국가수준 학업성취도평가 문제집 고2 영어 (2022년),NaN,['국내도서 > 고등학교참고서 > 고등-문제집 > 영어영역']
...,...,...,...
49824,김승봉 경찰헌법 기출문제집,2022,['국내도서 > 수험서/자격증 > 공무원 수험서 > 경찰공무원(승진) > 기타 과목']
49859,전국 영어/수학 학력 경시대회 수학 기출문제집 전기 중등 1학년,2021,"['국내도서 > 중학교참고서 > 중학경시대회', '국내도서 > 중학교참고서 > 중1..."
49890,적중 100 영어 기출문제집 중 3-1,"천재 정사열,2022",['국내도서 > 중학교참고서 > 중3-문제집 > 영어(중등3)']
49928,요양보호사 필기&실기문제집 7일만에 합격하기,2022,"사회과학 > 사회학, 사회문제 > 사회복지"


In [32]:
data[['제목','부제','분류','책소개','출판사리뷰']][data['제목'].str.contains('시험', na=False)]

,제목,부제,분류,책소개,출판사리뷰
43,2022 위생사 실기시험문제,개정16판,"기술과학 > 의학 > 위생학, 공공의학",이 책은 최근 개정된 새 법령과 출제 기준에 따라 집필하여 발행한 책이다. 각 과목...,- 하재남 교수의 누구도 따라올 수 없는 합격 노하우!!\n- 자주 출제되는 단원별...
218,2021 화훼장식기능사 필기시험 7일 완성 최종합격노트,2021년도 CBT 상시시험 완벽 대비(핵심이론 합격노트 + 기출문제 21회분 + ...,"예술 > 공예, 장식미술 > 목, 죽, 왕골공예",화훼장식기능사 D-DAY 7 합격프로젝트\n\n“7일이면 충분합니다!”\n\n202...,1. 시험에 꼭 나오는 핵심이론만 제대로 정리한 합격 수험서\n\n2020년 새로운...
424,버스운전자격시험 기출모의고사,"기출적중모의고사 8회분 수록,2019년 최신 기출문제 복원,2020",['국내도서 > 수험서/자격증 > 운전면허'],2019년 최신 기출문제를 복원 수록함으로써 버스운전자격시험을 준비하는 수험생들이 ...,본서는 2020 버스운전자격시험을 준비하는 수험생들이 최근 출제되는 기출유형을 파악...
478,농어촌개발 컨설턴트 자격시험 대비 문제집 1500,NaN,"기술과학 > 농업, 농학 > 농업경제",▶ 이 책은 농어촌개발 컨설턴트 자격시험 대비 문제에 대해 다룬 도서입니다. 농어촌...,NaN
515,"큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1, 2, 3급)상",2022,"['국내도서 > 수험서/자격증 > 한국사능력검정시험 > 심화(1,2,3급)']",2022 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화 상\n오랜 연구와 검증으...,큰별쌤 최태성의 별별한국사 한국사능력검정시험 시리즈가 압도적 1위인 이유\n1. 오...
...,...,...,...,...,...
49567,UNION 2023년도 제12회 변호사시험 기출문제집,공법·형사법·민사법: 사례·기록형,['국내도서 > 수험서/자격증 > 인문/사회/법(고등고시) > 변호사시험 > 헌법'...,"첫째, 사례형의 경우 - 집단토론과 반복검토를 통한 해설의 완성도를 극대화하였다. ...",NaN
49653,버스운전자격시험 3일만에 끝내기(8절),버스운전자격시험 대비 / 2021년 개정 완벽 반영 / 과목별 핵심문제와 실전모의고...,['국내도서 > 수험서/자격증 > 운전면허'],2021년 개정 법령을 완벽하게 반영해 전면 개정된 ‘버스운전자격시험 3일만에 끝내...,문제은행식에 최적화된 핵심 문제집으로 버스운전자격시험 단기 합격\n문제은행식으로 출...
49711,나합격 금속재료시험 열처리기능사 필기+실기+무료동영상,"2020,무료동영상 제공,나합격 수험생 지원센터 운영","기술과학 > 공학, 공업일반 > 공학, 공업일반",한 권으로 한 번에 합격하자!!\n나합격 100%만의 합격 비법\nSTEP 01 스...,한 권으로 한 번에 합격하자!!\n나합격 100%만의 합격 비법\nSTEP 01 스...
49858,한국사능력검정시험,"기본 4급.5급.6급,2020,메가스터디 단기폭발","['국내도서 > 수험서/자격증 > 한국사능력검정시험 > 기본(4,5,6급)']","■ 교재의 특징\n\n나올 것만 보고, 듣고, 풀다 \n\n√ 무료 개념 강의 제공...","나올 것만 보고, 듣고, 풀다 \n\n√ 무료 개념 강의 제공 : 한국사능력검정시험..."


In [ ]:
# 분류가 없는 것들은 대개 자격증 시험 관련 서적
data.loc[(data['분류'].isnull()),['제목','부제','저자','분류']]

,제목,부제,저자,분류
171,상표법 기출문제,변리사시험 1차 대비,김영남 지음,NaN
291,리담 디자인보호법,제10판,김인배 지음,NaN
293,4급 항해사 문제집,"해기사 자격시험,2021",김성곤 지음,NaN
451,600문제로 합격하는 수렵면허V3,2022,수렵문화연구회 지음,NaN
535,행정심판사례 연습()(2020),행정사 2차,정진석 지음,NaN
610,배극윤의 품질경영기사 필기,2022,배극윤 지음,NaN
615,물류관리사 벼락치기 핵심요약집,"개정 법률 등 개정사항 완벽 반영, 2022","박준혁, 백소라, 전표훈 지음",NaN
724,보건행정 기출문제집(보건직 공무원)(2020),2010~2019년 보건직 기출문제 엄선 수록,"권이승, 전봉재, 권영은 지음",NaN
794,전기기능사 필기(2020)(2주완성),CBT 완벽대비,김영복 지음,NaN
804,2023 중식 일식 복어 조리기능사 실기시험문제,개정판,"정수빈, 박선화 저 지음",NaN


In [ ]:
pd.reset_option("all")

<ipython-input-76-815ccf52250a>:1: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option("all")
<ipython-input-76-815ccf52250a>:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option("all")


### 저자
* 000 옮김 -> 해당 내용은 split해서 메타데이터에 집어넣기 or 삭제
* "저자" 변수에는 저자명만 들어가도록
* 그러나...
    * 편저,원저,지음,저,글,그림 등 오만가지 작가의 형태가 존재
    * 그냥 그대로 냅두는게 가장 나을 것 같음

In [ ]:
# def extract_author(Author_str):
#     parts = re.split(r'\s지음', Author_str)  # "지음"을 기준으로 split
#     return ' 지음'.join(parts[:-1]) if len(parts) > 1 else Author_str  # 마지막 "지음" 앞부분만 남김


# # 옮긴이
# def extract_translator(author_str):
#     parts = re.split(r'\s(지음|옮김|엮음)', author_str)
#     if "옮김" in parts:
#         idx = parts.index("옮김")
#         return parts[idx + 1].strip() if idx + 1 < len(parts) else None
#     return None  # 옮긴이가 없을 경우


# data['저자'] = data['저자'].apply(extract_author)
# data['저자'].head(50)

,저자
0,이민주(무궁화)
1,오가사와라 시게타카
2,"오현옥, 여동호, 김혜중, 우기숙, 이어은, 김진수, 임은희, 정해광"
3,조용석
4,크리스 반 알스버그
5,장한식
6,임홍택 저
7,편집부
8,홍정림
9,김종관


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
data['저자'][data['저자'].str.contains('편역', na=False)]

,저자
85,로버트 리 프로스트 저/신재실 편역
504,일연 저/임명현 편역
2407,이일환 편역
16218,"홍선영, 윤소영, 박미경, 복보경 편역"
16632,진윤영 편역
18428,허균 저/달상 그림/김영희 해설/설흔 편역
43744,송재소 편역
44425,알루보물레 스마나사라 저/장운갑 편역/강성욱 역
48478,앙투안 드 생텍쥐페리 원저/심재홍 편역
49113,김대현 편역


In [ ]:
data['저자'][data['저자'].str.contains('편저', na=False)]

,저자
200,SD적성검사연구소 편저
224,자격시험연구소 편저
545,주희 편저 / 이충구 역
905,SD적성검사연구소 편저
910,이지원 편저
983,"홍승한, 해커스 공인중개사시험 연구소 편저"
1389,백주기 편저
1471,자람교육연구소 편저
1646,SD법학연구소 편저
1743,남정선 편저


In [ ]:
data['저자'][data['저자'].str.contains('원저', na=False)]

,저자
423,몬마 츠카사 원저/시카코 글그림
454,어네스트 밀러 헤밍웨이 원저/이지훈 해설/강정규 역
725,그림 형제 원저/파란달 글그림
851,아머드 사우루스 원저/슈가캔디 글/김규태 그림
1022,에노모토 카이세이 원저/무로코이치 글그림/김동수 역
1143,"에구치 렌 원저, 후타바 모모 그림, 김현주"
1745,과앤 원저/밈바 글그림
3492,"이솝 원저, 박정미"
3803,나비코 원저/안즈테이 리코 글그림/문연주 역
4017,하무오 원저/텟타 엔지 글그림


In [ ]:
pd.reset_option('all')

<ipython-input-27-056908e7e3ef>:1: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
<ipython-input-27-056908e7e3ef>:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


In [ ]:
# 메타 데이터에 들어가야할 columns = ['ISBN','ITEM_ID','BID','GOODS_NO','원제','발행자','발행일','페이지','가격','표지','INSERT_DATE','UPDATE_DATE']
# 삭제/통합 시켜야할 columns = ['부제','원제','']
# 분류가 null에 해당하는 책은 대다수 자격증 관련 도서
# 제목이 없는 책은 부제를 제목으로 봐도 무방함.
# 제목과 부제가 모두 존재하는 책은 제목을 제목_부제 형식으로 통합시키는게 좋을 것 같고, 부제가 없는 책은 제목 그대로 냅두기. 원제는 메타 데이터에 합치되 실제 모델에서는 사용될 일이 없을 것 같음
# 저자에서 , 000 옮김 형태는 따로 split을 활용하여 뽑아낸 뒤, metadata에 포함시키거나 삭제
# 실질적인 VERTOR DB Documents 에 해당되는 핵심 columns = ['분류','제목','저자','책 소개','저자소개','목차','출판사리뷰']


# 간략소개와 책 소개
# 책소개가 없는데 간략소개만 있는 경우, 간략 소개라도 책 소개로 첨부


# 부제와 제목 모두 null이 아닌 경우에만 data['제목']과 data['부제']를 data['제목']의 한 컬럼으로 합치는데 form은 다음과 같음
# str('data['제목']'+ '_' + 'data['부제']')
# data['제목']이 없는 경우, data['부제']를 data['제목']으로 가져오기

### 제목 & 부제
* "제목"과 "부제"가 NotNull이면, "제목_부제" 형태로 변환
* "부제"가 Null 이면, "제목"
* "제목"이 Null 이면, "부제"가 "제목"으로
* "원제"는 메타데이터 or 삭제

#### 부제 전처리

In [ ]:
data[['제목','부제']].loc[data['제목']=='성경전서']

,제목,부제
4615,성경전서,"단본,색인,개역한글,큰글씨"
5837,성경전서,"새찬송가,큰글씨,NKR73BU,색인"
12583,성경전서,"한글판,단본,색인,72HC"
13317,성경전서,"새찬송가,NKR73EBU"
17854,성경전서,독일관주·해설
26144,성경전서,"색인,개역한글,큰글씨"
27769,성경전서,개역한글
38477,성경전서,"개역한글,62HB,단본"
42087,성경전서,"단본색인,62HC,한글판"
43668,성경전서,"단본,색인,개역한글,큰글씨"


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# 1095, 1909, 2098, 2373, 2830, 3171, 3341, 4496, 4589, 4615

# 큰글씨책- 이라는 표현이 도움이 돼?

In [ ]:
# 부제에서 NK~,단본,합본,무색인,색인,단본색인,단본무색인,합본색인,합본무색인,발송,RN~,72HC~ 삭제 필요

# 정규식 패턴 설정
pattern = r'(NKR\d*|NKS\d*|RN\d*|72HC\d*|단본|합본|무색인|색인|단본색인|단본무색인|합본색인|합본무색인|발송|한글|글씨)'

# 조건 필터링
second_title = data[data['부제'].str.contains(pattern, regex=True, na=False)]

second_title[['제목','부제']]

<ipython-input-30-cbd15fb4fab5>:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  second_title = data[data['부제'].str.contains(pattern, regex=True, na=False)]


,제목,부제
223,기획서 마스터,"제안 통과까지 프로 일잘러들의 실전 스킬,아이디어에서 기획서 작성,큰글씨책"
310,진자앙 시선,큰글씨책
393,신나는 한글여행 8단계,유아 한글의 모든것
520,원서발췌 아녀영웅전,큰글씨책
525,The Advanced Ventilator Book,한글판
540,우리말을 찾아라,한글 사전을 만든 사람들
875,계안록,큰글씨책
1095,쉽게 간추린 취리히성경해설,"단본,무색인,NKS73XSZU"
1209,"소암, 바람의 노래","큰글씨책,손선영 장편소설"
1374,허백당집,"큰글씨책, 10% 원서발췌"


##### 중복값 처리

In [ ]:
c = data.groupby('제목').filter(lambda x: len(x) > 1).sort_values('제목')

In [ ]:
c = c.drop(columns=['ISBN','ITEM_ID','BID','GOODS_NO','발행일','페이지','가격','INSERT_DATE','UPDATE_DATE'])

In [ ]:
# 제목이 중복되는 애들 중 부제가 없는 애들을 탈락
# 제목은 같은데 저자가 다른 경우
# 제목은 같은데 부제가 다른 경우
# 부제가 숫자 년도인 경우인 애들

# 중복인 값들 중 제목과 부제가 같은 애들 중 저자가 다른 애들 먼저 뽑아보고 문제가 없다 싶으면, 제목 부제 같은 애들 drop

In [ ]:
cols = ['분류', '부제', '간략소개', '책소개', '저자소개', '목차', '출판사리뷰']


data.groupby('제목').filter(lambda x: x[cols].nunique().max() > 1).sort_values('제목')

,ISBN,ITEM_ID,BID,GOODS_NO,분류,제목,부제,원제,저자,발행자,...,페이지,가격,표지,간략소개,책소개,저자소개,목차,출판사리뷰,INSERT_DATE,UPDATE_DATE
12290,9791190397124,303075254,NaN,114665638.0,"[""국내도서 > 잡지 > 교양/문예/인문 > 문예지"", ""국내도서 > 잡지 > 교양...",1.5℃ (1.5도씨) ISSUE : No.4 CLIMATE GOURMET [2022],None,None,"소울에너지, 볼드 피리어드 저 지음",볼드피리어드,...,256,16000,https://image.aladin.co.kr/product/30307/52/co...,인류 생존의 마지노선 1.5℃를 지키기 위한 기후 위기 대응 매거진 「1.5℃」「1...,인류 생존의 마지노선 1.5℃를 지키기 위한 기후 위기 대응 매거진 「1.5℃」\n...,저 : 소울에너지\n‘소울 에너지 (SOUL ENERGY)’는 미래 환경을 생각하며...,기후 위기 시대 음식의 초상<br/>식탁을 엎어야 할 때<br/>우리는 기후를 먹고...,"2021년 UN식량농업기구(FAO)가 공개한 자료에 따르면, 인간 활동으로 발생하는...",2022-12-28T14:09:04.000Z,None
26639,9791198296245,316123479,NaN,114665638.0,"[""국내도서 > 잡지 > 교양/문예/인문 > 문예지"", ""국내도서 > 잡지 > 교양...",1.5℃ (1.5도씨) ISSUE : No.4 CLIMATE GOURMET [2022],None,None,"소울에너지, 걷는사람 저 지음",소울에너지,...,256,16000,https://image.aladin.co.kr/product/31612/34/co...,인류 생존의 마지노선 1.5℃를 지키기 위한 기후 위기 대응 매거진 「1.5℃」「1...,인류 생존의 마지노선 1.5℃를 지키기 위한 기후 위기 대응 매거진 「1.5℃」\n...,저 : 소울에너지\n‘소울 에너지 (SOUL ENERGY)’는 미래 환경을 생각하며...,기후 위기 시대 음식의 초상\n식탁을 엎어야 할 때\n우리는 기후를 먹고 위기를 남...,"2021년 UN식량농업기구(FAO)가 공개한 자료에 따르면, 인간 활동으로 발생하는...",2024-03-29T05:17:25.000Z,None
41919,9791190563666,183607373,16395190.0,NaN,"[""국내도서 > 고등학교참고서 > 고등-문제집 > 영어영역""]",100발 100중 고등 영어 B 기출문제집,비상 홍민표,None,백발백중 편집부 지음,에듀원,...,156,12000,https://image.aladin.co.kr/product/18360/73/co...,"1. 교과서 핵심 개념 학습 : 단원별 주요 어휘와 어구, 의사소통기능 표현, 문법...","1. 교과서 핵심 개념 학습 : 단원별 주요 어휘와 어구, 의사소통기능 표현, 문법...",None,Lesson 03 Contribute \nLesson 04 Explore,리뷰〉\n1. 기출문제 종합 분석 \n- 기출문제를 종합 분석하여 출제율이 높은 문...,2022-03-22T15:41:51.000Z,None
6602,9791190563703,183603704,16389856.0,NaN,"[""국내도서 > 고등학교참고서 > 고등-문제집 > 영어영역""]",100발 100중 고등 영어 B 기출문제집,"능률 양현권,2020",None,백발백중편집부 지음,에듀원,...,160,12000,https://image.aladin.co.kr/product/18360/37/co...,"1. 교과서 핵심 개념 학습 : 단원별 주요 어휘와 어구, 의사소통기능 표현, 문법...","1. 교과서 핵심 개념 학습 : 단원별 주요 어휘와 어구, 의사소통기능 표현, 문법...",None,"Lesson 03 Take Action, Make a Difference \nLes...",1. 기출문제 종합 분석 \n- 기출문제를 종합 분석하여 출제율이 높은 문제 다량 ...,2022-03-22T15:41:47.000Z,None
10453,9791165753825,288869086,21511253.0,NaN,"[""국내도서 > 고등학교참고서 > 고등-문제집 > 영어영역""]",100발 100중 고등 영어 B 기출문제집,"고등 내신 1등급을 위한 기출문제집,YBM 한상호,2022",None,에듀원 편집부 지음,에듀원,...,224,13000,https://image.aladin.co.kr/product/28886/90/co...,기출문제를 종합 분석하여 출제율이 높은 문제를 다량 수록하였다. 출제 경향과 출제율...,기출문제를 종합 분석하여 출제율이 높은 문제를 다량 수록하였다. 출제 경향과 출제율...,None,"Lesson 03 Eat Right, Be Healthy\nSTEP 1. Words...",1) 기출문제 종합 분석\n- 기출문제를 종합 분석하여 출제율이 높은 문제 다량 수...,2022-03-22T17:47:49.000Z,2022-05-01T04:00:16.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42244,9791158065263,274235693,20666429.0,NaN,"[""국내도서 > 고등학교참고서 > 논술/면접/수시 > 입학사정관/자기소개서""]",훔쳐서라도 보고 싶은 대입 자소서,"학종 금수저로 가는 컨설팅의 종결판,2022",None,"신동훈, 김민지 지음",골든벨,...,166,15000,https://image.aladin.co.kr/product/27423/56/co...,2022년 학생부종합전형에 주목해야 하는 이유!\n입학사정관이 원하고 대입전문코치가...,2022년 학생부종합전형에 주목해야 하는 이유!\n\n입학사정관이 원하고 대입전문코...,신동훈\n저자 : 신동훈\n■ 지원자가 말하고 싶은 내용을 논리적으로 표현할 수 있...,Chapter 1 한 눈에 보는 2022 대입 트렌드\r\n지금은 학생부종합전형 시...,■ 대입 자소서는 입학사정관을 설득하는 글이다.\n■ 대입 자소서는 실제로 ‘내’가...,2022-03-22T17:20:54.000Z,None
17038,9788967819460,296820889,22528803.0,NaN,"[""국내도서 > 유아 > 4~7세 > 그림책"", ""국내도서 > 유아 > 그림책 > ...",흥부와 놀부,None,None,김미혜 지음 / 최예리 일러스트,애플비북스,...,36,5500,https://image.aladin.co.kr/product/29682/8/cov...,착한 동생 흥부를 쫓아낸 못된 형 놀부는 어떻게 됐을까요? \r달라도 너무 다른 두...,착한 동생 흥부를 쫓아낸 못된 형 놀부는 어떻게 됐을까요?\n달라도 너무 다른 두 ...,None,None,1. 할머니가 말해 주듯 말맛을 제대로 살린 옛이야기를 만나요.\n이불 하나에 자식...,2022-08-23T02:43:05.000Z,None
14823,9788966191130,280312156,20953704.0,NaN,"[""국내도서 > 유아 > 4~7세 > 그림책"", ""국내도서 > 유아 > 그림책 > ...",흥부와 놀부,희망을 담은 이야기,None,민정원 지음 / 이수인 일러스트,인북,...,30,10000,https://image.aladin.co.kr/product/28031/21/co...,가난하지만 착하고 어진 흥부는 새끼 제비의 다리를 고쳐 주고 박씨를 얻어요.흥부가 ...,가난하지만 착하고 어진 흥부는 새끼 제비의 다리를 고쳐 주고 박씨를 얻어요.\n흥부...,None,이 도서는 목차가 없습니다,None,2022-03-24T02:31:44.000Z,None
31206,9788998441081,244151572,16395543.0,NaN,"[""국내도서 > 소설/시/희곡 > 한국소설 > 2000년대 이후 한국소설""]",희망,양귀자 장편소설,None,양귀자 지음,쓰다

In [ ]:
data['제목'].duplicated().value_counts()

,count
제목,
False,48401
True,1599


In [ ]:
pd.reset_option('all')

<ipython-input-88-056908e7e3ef>:1: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
<ipython-input-88-056908e7e3ef>:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


In [ ]:
data['제목'] = data.apply(lambda x:
                         f"{x['제목']}_{x['부제']}" if pd.notna(x['제목']) and pd.notna(x['부제'])
                         else x['부제'] if pd.isna(x['제목']) and pd.notna(x['부제'])
                         else x['제목'], axis=1)

print(data[['제목', '부제']].head(10))

                                                  제목  \
0                인생에서 정지 버튼을 누르고 싶었던 순간들_마이 페이보릿 시퀀스   
1  그림과 실습으로 배우는 도커 & 쿠버네티스_개념과 작동 원리가 쏙쏙 이해되는 완벽 입문서   
2              책에 나를 바치다_책을 통해 변화하는 나, 글을 쓰며 성장하는 우리   
3                     조용석 알파로직 경찰학 동형모의고사_2022 2차 대비   
4                                    세상에서 가장 맛있는 무화과   
5                      한일 고대사의 재건축1_왜(倭)와 임나(任那)의 진실   
6                         그건 부당합니다_Z세대 공정의 기준에 대한 탐구   
7        내신콘서트 중학 중간+기말고사 기출문제집 영어 중3-1_미래엔 최연희,2020   
8                                  실업과 건강_정책연구 21-01   
9                                              부부 시계   

                            부제  
0                  마이 페이보릿 시퀀스  
1    개념과 작동 원리가 쏙쏙 이해되는 완벽 입문서  
2  책을 통해 변화하는 나, 글을 쓰며 성장하는 우리  
3                   2022 2차 대비  
4                         None  
5             왜(倭)와 임나(任那)의 진실  
6            Z세대 공정의 기준에 대한 탐구  
7                 미래엔 최연희,2020  
8                   정책연구 21-01  
9                         None  


In [ ]:
data['제목'].isnull().unique()
# False

array([False])

### 책 소개 & 간략 소개
* 중복 columns 이라면, 간략 소개 drop

### 책소개 == 간략소개?
* 중복 columns 에 대한 검증이 필요 - Cosine Similarity

In [ ]:
## \n 이슈로 동일 내용이나 다른 값으로 나옴

data[['책소개','간략소개']].loc[data['간략소개'] != data['책소개']].head(30)

,책소개,간략소개
0,"“잠깐 정지! 저 장면 완전 내 이야기 아니야?”\n\n〈리틀 포레스트〉, 〈소공녀...","“잠깐 정지! 저 장면 완전 내 이야기 아니야?”\n〈리틀 포레스트〉, 〈소공녀〉,..."
1,컨테이너나 도커를 도통 이해하기 어려운 분들을 위한 본격 도커 입문서!\n이 책은 ...,컨테이너나 도커를 도통 이해하기 어려운 분들을 위한 본격 도커 입문서!이 책은 컨테...
2,프롤로그: 누구나 글을 쓰는 시대에 살고 있다\n\n누구나 글을 쓰는 시대에 살고 ...,프롤로그: 누구나 글을 쓰는 시대에 살고 있다\n누구나 글을 쓰는 시대에 살고 있다...
3,10회 모의고사로 이루어진\n2022년 2차 시험대비 조용석 경찰학 동형모의고사\n...,10회 모의고사로 이루어진 2022년 2차 시험대비 조용석 경찰학 동형모의고사※ 2...
4,무엇을 꿈꾸든 그것이 현실이 된다?\n아주 특별한 무화과 두 개가 빚어내는\n꿈과 ...,무엇을 꿈꾸든 그것이 현실이 된다? 아주 특별한 무화과 두 개가 빚어내는 꿈과 현실...
5,한일 역사전쟁의 종지부를 찍기 위해\n\n오염되고 왜곡된 임나와 일본부의 진짜 얼굴...,한일 역사전쟁의 종지부를 찍기 위해\n오염되고 왜곡된 임나와 일본부의 진짜 얼굴을 ...
6,"“우리가 믿어온 ‘공정’의 기준이 뒤집어진다!”\n『90년생이 온다』 이후 4년, ...","“우리가 믿어온 ‘공정’의 기준이 뒤집어진다!”『90년생이 온다』 이후 4년, 그 ..."
7,- 독자대상 : 중학교 3학년\n\n- 구성 : 기출문제집\n\n- 특징 : 1학기...,- 독자대상 : 중학교 3학년\n- 구성 : 기출문제집\n- 특징 : 1학기 전과정...
9,자연 그대로의 성찰로 삶의 희망을 노래하다\n김종관 시인의 첫 시집 『부부 시계』가...,자연 그대로의 성찰로 삶의 희망을 노래하다김종관 시인의 첫 시집 『부부 시계』가 ‘...
10,입사 2년차에 돌입! 수많은 「처음」 때문에 약동!\n\n네크로그란트 흑마법사에 입...,입사 2년차에 돌입! 수많은 「처음」 때문에 약동!네크로그란트 흑마법사에 입사한 지...


In [ ]:
import re

a = data['책소개'].astype(str)
b = data['간략소개'].astype(str)

# \n 전부 제거
a_clean = a.apply(lambda x: re.sub(r'\n+', '', x).strip())
b_clean = b.apply(lambda x: re.sub(r'\n+', '', x).strip())

# 토씨하나까지 동일한 텍스트인가?
exact_match_count = (a_clean == b_clean).sum()

print(f"토씨하나까지 동일한 텍스트 : {exact_match_count}")

토씨하나까지 동일한 텍스트 : 10386


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch

# 임베딩에 gpt
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# 토씨하나까지 동일한 텍스트 drop
mask = a_clean != b_clean
a_filtered = a_clean[mask].reset_index(drop=True)
b_filtered = b_clean[mask].reset_index(drop=True)

# Sentence Transformer - Cosine Similarity
model = SentenceTransformer('paraphrase-MiniLM-L6-v2', device=device)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print(len(a_filtered))
print(len(b_filtered))

39614
39614


In [ ]:
# 임베딩 32batch size
a_embeddings = np.array(model.encode(a_filtered.tolist(), batch_size=32, show_progress_bar=True))
b_embeddings = np.array(model.encode(b_filtered.tolist(), batch_size=32, show_progress_bar=True))

# 코사인 유사도 계산 (dense_output=False 추가)
cos_similarities = np.diag(cosine_similarity(a_embeddings, b_embeddings, dense_output=False))

# 결과를 데이터프레임으로 저장
x = pd.DataFrame({
    '책소개(전처리)': a_filtered,
    '간략소개(전처리)': b_filtered,
    'Cosine Similarity': cos_similarities
})

Batches:   0%|          | 0/1238 [00:00<?, ?it/s]

Batches:   0%|          | 0/1238 [00:00<?, ?it/s]

In [ ]:
## 대부분 한 쪽이 Null인 경우, Cosine Similarity가 낮음

x[x['Cosine Similarity'] <= 0.8]

## 둘 다 Null이 아니라면??

,책소개(전처리),간략소개(전처리),Cosine Similarity
73,None,이재수 실기,0.220732
80,None,라푼젤,0.217411
176,None,전기기사·전기산업기사 기사실기특강,0.201585
184,성경을 바로 설교하려면 성경 저자의 의도를 아는 것이 아주 중요하다. 설교를 잘하는...,None,0.230660
189,None,소방법령 1 핵심정리,0.234012
...,...,...,...
39497,None,수능 마무리 봉투모의고사 파이널 테스트 국어영역 화법과 작문 (2023년),0.234096
39561,None,관공서중심의 공공디자인복합건축물,0.216924
39572,None,Vaccine 1. 성인간호학 기출문제풀이,0.115689
39582,None,운동 역학,0.249642


In [ ]:
# 둘 다 Null이 아닌 경우
filtered_x = x[(x['Cosine Similarity'] <= 0.8) & ((x['책소개(전처리)'] != 'None') & (x['간략소개(전처리)'] != 'None'))]
print(len(filtered_x))

19


In [ ]:
# 19개 값들 역시 영문이거나 특수기호 차이 or 대부분 비슷함.
filtered_x

,책소개(전처리),간략소개(전처리),Cosine Similarity
8824,This book is the first of a three-part series ...,This book is the first of a three-part series ...,0.718653
10857,"Easily, Happily, and Simply from Today!Startin...","Easily, Happily, and Simply from Today!Startin...",0.790483
10898,★ ★ ★?Wonderful activity book with easy colori...,★ ★ ★?Wonderful activity book with easy colori...,0.771631
16624,"Getting Married to Someone I Met A Month Ago, ...","Getting Married to Someone I Met A Month Ago, ...",0.706693
17473,가족이란 무엇일까? 그럼 단란한 가정은 어떤 모습일까? 글을 적고 난 뒤 고요한 돈...,격동기 속에서 가족의 의미....?,0.772124
20023,이 책은 코딩마법서 C/C++ 1권 STONE 이후의 다음 버전 책입니다. 1권이 ...,많은 성원 부탁드립니다.이 책의 시리즈HTTP://WWW.KYOBOBOOK.CO.K...,0.481626
20726,We proudly present to you our newest&nbsp;Line...,We proudly present to you our newestLined Note...,0.749217
21341,EVERYDAYI LOOK AT THE MIRRORWONDERING WHO’S TH...,EVERYDAY I LOOK AT THE MIRROR WONDERING WHO’S ...,0.651123
25231,숨 막히는 코로나 시대에 떠난담양에서의 나 홀로 체류형한 달 살기 여행이야기코로나 ...,숨 막히는 코로나 시대에 떠난 담양에서의 나 홀로 체류형 한 달 살기 여행이야기코로...,0.776060
25860,"우리가 모르는 어딘가, 시간이 멈춰 버린 곳, \r영원한 밤의 세상이 존재한다면? ...","우리가 모르는 어딘가, 시간이 멈춰 버린 곳, \r영원한 밤의 세상이 존재한다면? ...",0.682300


In [ ]:
index_list = filtered_x.index.tolist()
print(index_list)

[8824, 10857, 10898, 16624, 17473, 20023, 20726, 21341, 25231, 25860, 26041, 27784, 30294, 31497, 33305, 34231, 34280, 36109, 36849]


### "책소개"가 Null이고 "간략소개"가 NotNull이라면, "간략소개"로 "책소개"를 대체

In [ ]:
# 책 소개가 null인데, 간략 소개가 notnull인 행은 "책 소개"를 "간략 소개"로 대체하기

data['책소개'] = data.apply(lambda x: x['간략소개'] if pd.isna(x['책소개']) and pd.notna(x['간략소개']) else x['책소개'], axis=1)

data[['책소개', '간략소개']].head(10)

,책소개,간략소개
0,"“잠깐 정지! 저 장면 완전 내 이야기 아니야?”\n\n〈리틀 포레스트〉, 〈소공녀...","“잠깐 정지! 저 장면 완전 내 이야기 아니야?”\n〈리틀 포레스트〉, 〈소공녀〉,..."
1,컨테이너나 도커를 도통 이해하기 어려운 분들을 위한 본격 도커 입문서!\n이 책은 ...,컨테이너나 도커를 도통 이해하기 어려운 분들을 위한 본격 도커 입문서!이 책은 컨테...
2,프롤로그: 누구나 글을 쓰는 시대에 살고 있다\n\n누구나 글을 쓰는 시대에 살고 ...,프롤로그: 누구나 글을 쓰는 시대에 살고 있다\n누구나 글을 쓰는 시대에 살고 있다...
3,10회 모의고사로 이루어진\n2022년 2차 시험대비 조용석 경찰학 동형모의고사\n...,10회 모의고사로 이루어진 2022년 2차 시험대비 조용석 경찰학 동형모의고사※ 2...
4,무엇을 꿈꾸든 그것이 현실이 된다?\n아주 특별한 무화과 두 개가 빚어내는\n꿈과 ...,무엇을 꿈꾸든 그것이 현실이 된다? 아주 특별한 무화과 두 개가 빚어내는 꿈과 현실...
5,한일 역사전쟁의 종지부를 찍기 위해\n\n오염되고 왜곡된 임나와 일본부의 진짜 얼굴...,한일 역사전쟁의 종지부를 찍기 위해\n오염되고 왜곡된 임나와 일본부의 진짜 얼굴을 ...
6,"“우리가 믿어온 ‘공정’의 기준이 뒤집어진다!”\n『90년생이 온다』 이후 4년, ...","“우리가 믿어온 ‘공정’의 기준이 뒤집어진다!”『90년생이 온다』 이후 4년, 그 ..."
7,- 독자대상 : 중학교 3학년\n\n- 구성 : 기출문제집\n\n- 특징 : 1학기...,- 독자대상 : 중학교 3학년\n- 구성 : 기출문제집\n- 특징 : 1학기 전과정...
8,이 책은 한국노동연구원의『실업과 건강』을 다룬 정부간행물입니다.,이 책은 한국노동연구원의『실업과 건강』을 다룬 정부간행물입니다.
9,자연 그대로의 성찰로 삶의 희망을 노래하다\n김종관 시인의 첫 시집 『부부 시계』가...,자연 그대로의 성찰로 삶의 희망을 노래하다김종관 시인의 첫 시집 『부부 시계』가 ‘...


In [ ]:
# "책소개"와 "간략소개" 둘 다 Null인 값들 1351개
data['책소개'].isnull().sum()

1351

In [ ]:
# "제목"과 합쳐진 "부제" & "책소개"와 합쳐진 "간략소개" drop
data = data.drop(columns=['부제','간략소개'])

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  object 
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           46967 non-null  object 
 5   제목           50000 non-null  object 
 6   원제           4506 non-null   object 
 7   저자           50000 non-null  object 
 8   발행자          50000 non-null  object 
 9   발행일          50000 non-null  object 
 10  페이지          50000 non-null  int64  
 11  가격           50000 non-null  int64  
 12  표지           50000 non-null  object 
 13  책소개          48649 non-null  object 
 14  저자소개         37934 non-null  object 
 15  목차           45459 non-null  object 
 16  출판사리뷰        29156 non-null  object 
 17  INSERT_DATE  50000 non-null  object 
 18  UPDATE_DATE  10000 non-null  object 
dtypes: f

### 가격 & 페이지

In [ ]:
data['페이지'].max(),data['페이지'].min()

(12839481, 0)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
data[['제목','페이지','가격']].loc[data['페이지']>=3000]

## 전집이 아닌

,제목,페이지,가격
25,"천룡팔부 세트_김용 대하역사무협,전10권",4000,128000
1126,"스토리텔링성경 구약 Special edition_성경 전 장을 이야기로 풀어 쓴,모...",3499,110000
1212,수학도둑 61~91 세트_코믹 메이플스토리,5040,269600
1601,법전_2022,6000,125000
1924,이문열 삼국지 세트_전10권,3944,148000
3301,"노동법편람_2021,법 령 규칙 대사식",3120,120000
3452,"여래 이삼한, 인류 최고 깨달음의 진리 세트",7232,600000
3623,독립운동가 100인 만화 프로젝트 제1차 위대한 시민의 역사 세트_전33권,9900,495000
6367,리비우스 로마사 세트_전4권,3140,110000
9397,"회복역 성경_무색인,주석",3264,137000


In [ ]:
# 제목/페이지/가격
# 밀리몰리맨디 이야기 2_깜짝 선물을 받아요	188180	12000
# 혐오의 시대, 철학의 응답_모욕당한 자들의 반격을 위한 언어를 찾아서	9626460	9800
# 군무원 15일 완성 행정직_국방부 및 육군·해군·공군·해병대 9급 군무원 채용 대비...	13350	15000
# 20세기 공간 이론_Spatial Theories in the Twentieth C...	28000	15000
# 인형의 집	9800	9800
# 자동차 전기장치(신편)	20000	20000
# 교사, 다시 뛰자!_교사가 행복해야 아이들이 산다	12839481	9800

In [ ]:
data[['제목','페이지','가격']].loc[(data['가격'] == data['페이지'])]

,제목,페이지,가격
40612,인형의 집,9800,9800
40874,자동차 전기장치(신편),20000,20000


In [ ]:
data[['제목','페이지','가격']].loc[(data['페이지'] == 0)]

,제목,페이지,가격
30,기계설계_11판,0,37000
309,2023 박문각 공인중개사 기본서 1차 세트_제34회 공인중개사 시험 대비,0,74000
339,"비정형데이터분석(2학기, 워크북포함)",0,11000
352,독해력 자신감 초등 국어 2단계 (2022년)_학습 능력을 키우는 친절한 독해 훈련서,0,11000
443,IFRS 중급회계 하_제5판,0,15000
659,"교과교재연구및지도법(2학기, 워크북포함)",0,21400
739,오늘 갑자기 신이 원망스러운 사람에게_어느 오후 스쳐지나는 바람이 들려주는 이야기,0,15000
964,EBSi 강의노트 수능개념 과학탐구 생명과학2의 주연은 너! (2023년)_2024...,0,7500
1246,2023 권규호 공무원 국어 독해 강훈련,0,18000
1304,전국 영어/수학 학력 경시대회 영어 기출문제집 중2,0,17000


In [ ]:
data[['제목','페이지','가격']].loc[(data['가격'] <= 30000) & (data['페이지'] >= 1000)]

,제목,페이지,가격
272,2023 완벽대비 과년도 출제문제 중심 에너지관리기사 필기,1012,30000
1095,"쉽게 간추린 취리히성경해설_단본,무색인,NKS73XSZU",1536,20000
1291,UN에 취업하기,1492,16000
1913,"본능적인 그대 세트_이달아 장편소설,전2권",1088,27600
3079,"성서원 어드벤처 컬러 쉬운말성경 D브라운_읽는 대로 쏙쏙 이해되는, 일반 PU",1712,25000
3341,"몽골어-한국어 대조성경_성경전서, NKMG82DI, 단본",2096,25000
3442,정역학_기초부터 쉽게 배우는,2360,19000
4397,한자능력검정시험 1급(2020)_한국어문회 지정지침서,1040,29000
4615,"성경전서_단본,색인,개역한글,큰글씨",1755,26000
5297,굿데이 군인성경 (특초미니 합본/해설새찬송가/지퍼/PU/반달 색인/주석/국방색)_지...,1760,26000


In [ ]:
# 28241, 33901, 34820, 37747, 40612, 40874, 44538
# 해당 인덱스번호 페이지는 알수없음(0)
a = [28241, 33901, 34820, 37747, 40612, 40874, 44538]
data.loc[data.index.isin(a),'페이지'] = 0

In [ ]:
data.loc[data.index.isin(a),'페이지']

,페이지
28241,0
33901,0
34820,0
37747,0
40612,0
40874,0
44538,0


In [ ]:
pd.reset_option('all')

<ipython-input-43-056908e7e3ef>:1: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
<ipython-input-43-056908e7e3ef>:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


In [ ]:
print(f"최대 가격 : {data['가격'].max()}, 최소 가격 : {data['가격'].min()}")
print(f"최대 페이지지 : {data['페이지'].max()}, 최소 페이지 : {data['페이지'].min()}")

최대 가격 : 980000, 최소 가격 : 1000
최대 페이지지 : 20160, 최소 페이지 : 0


In [ ]:
data['가격']

,가격
0,15000
1,28000
2,16000
3,18000
4,13000
...,...
49995,10000
49996,13000
49997,11800
49998,5000


In [ ]:
data['페이지']

,페이지
0,208
1,400
2,280
3,184
4,32
...,...
49995,147
49996,192
49997,112
49998,176


In [ ]:
# 각 범위에 맞게 메모리 조정
data['가격'] = data['가격'].astype('int32')
data['페이지'] = data['페이지'].astype('int16')

In [ ]:
data['가격']

,가격
0,15000
1,28000
2,16000
3,18000
4,13000
...,...
49995,10000
49996,13000
49997,11800
49998,5000


In [ ]:
data['페이지']

,페이지
0,208
1,400
2,280
3,184
4,32
...,...
49995,147
49996,192
49997,112
49998,176


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  object 
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           46967 non-null  object 
 5   제목           50000 non-null  object 
 6   원제           4506 non-null   object 
 7   저자           50000 non-null  object 
 8   발행자          50000 non-null  object 
 9   발행일          50000 non-null  object 
 10  페이지          50000 non-null  int16  
 11  가격           50000 non-null  int32  
 12  표지           50000 non-null  object 
 13  책소개          48649 non-null  object 
 14  저자소개         37934 non-null  object 
 15  목차           45459 non-null  object 
 16  출판사리뷰        29156 non-null  object 
 17  INSERT_DATE  50000 non-null  object 
 18  UPDATE_DATE  10000 non-null  object 
dtypes: f

### 발행자 & 발행일

In [ ]:
data['발행자']

,발행자
0,21세기북스
1,위키북스
2,행복에너지
3,용감한북스
4,미래아이
...,...
49995,문학공원
49996,밥북
49997,세개의소원
49998,학산문화사


In [ ]:
# 발행일에 시간이 중요한가?
# T15:00:00.000Z : UTC 기준 시간정보
data['발행일']

,발행일
0,2020-05-12T15:00:00.000Z
1,2022-04-04T15:00:00.000Z
2,2020-06-24T15:00:00.000Z
3,2022-06-27T15:00:00.000Z
4,2021-12-09T15:00:00.000Z
...,...
49995,2020-08-24T15:00:00.000Z
49996,2021-10-20T15:00:00.000Z
49997,2020-02-09T15:00:00.000Z
49998,2020-02-24T15:00:00.000Z


In [ ]:
data['발행일'] = data['발행일'].str.split('T').str[0]
data['발행일'] = pd.to_datetime(data['발행일']).dt.date # 그냥 object로 냅두기

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  object 
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           46967 non-null  object 
 5   제목           50000 non-null  object 
 6   원제           4506 non-null   object 
 7   저자           50000 non-null  object 
 8   발행자          50000 non-null  object 
 9   발행일          50000 non-null  object 
 10  페이지          50000 non-null  int16  
 11  가격           50000 non-null  int32  
 12  표지           50000 non-null  object 
 13  책소개          48649 non-null  object 
 14  저자소개         37934 non-null  object 
 15  목차           45459 non-null  object 
 16  출판사리뷰        29156 non-null  object 
 17  INSERT_DATE  50000 non-null  object 
 18  UPDATE_DATE  10000 non-null  object 
dtypes: f

In [ ]:
save_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book_preprocessing_1.csv"

# CSV
data.to_csv(save_path, encoding='utf-8', index=False)